# Meraki Python SDK Demo: Offline Switch Finder

*This notebook demonstrates using the Meraki Python SDK to create a list of all the switches in a network that are offline.*

---

>NB: Throughout this notebook, we will print values for demonstration purposes. In a production Python script, the coder would likely remove these print statements to clean up the console output.

In this first cell, we install and import the required meraki and os modules, and open the Dashboard API connection using the SDK. Make sure you have [set up your environment variables first](https://github.com/meraki/dashboard-api-python/blob/master/notebooks/notebooksReadme.md#setting-up-your-environment-variables).

In [ ]:
# Install the relevant modules. If you are using a local editor (e.g. VS Code, rather than Colab) you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK.
%pip install meraki
%pip install tablib

# If you are using Google Colab, please ensure you have set up your environment variables as linked above, then delete the two lines of ''' to activate the following code:
'''
%pip install colab-env -qU
import colab_env
'''

# Rely on meraki SDK, os, and tablib -- more on tablib later
import meraki
import os
import tablib

# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# Setting API key this way, and storing it in the env variables, lets us keep the sensitive API key out of the script itself
# The meraki.DashboardAPI() method does not require explicitly passing this value; it will check the environment for a variable
# called 'MERAKI_DASHBOARD_API_KEY' on its own. In this case, API_KEY is shown simply as an reference to where that information is
# stored.
API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI()

Let's make a basic pretty print formatter, `printj()`. It will make reading the JSON later a lot easier, but won't be necessary in production scripts.

In [ ]:
def printj(ugly_json_object):
    
    # The json.dumps() method converts a JSON object into human-friendly formatted text
    pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)

    return print(pretty_json_string)

Most API calls require passing values for the organization ID and/or the network ID. In this second cell, we fetch a list of the organizations the API key can access, then pick the first org in the list, and the first network in that organization, to use for later operations. You could re-use this code presuming your API key only has access to a single organization, and that organization only contains a single network. Otherwise, you would want to review the organizations object declared and printed here to review its contents.

In [ ]:
# Let's make it easier to call this data later
# getOrganizations will return all orgs to which the supplied API key has access
organizations = dashboard.organizations.getOrganizations()
print('Organizations:')
printj(organizations)

# This example presumes we want to use the first organization as the scope for later operations. 
firstOrganizationId = organizations[0]['id']
firstOrganizationName = organizations[0]['name']

# Print a blank line for legibility before showing the firstOrganizationId
print('')
print(f'The firstOrganizationId is {firstOrganizationId}, and its name is {firstOrganizationName}.')

This example presumes we want to use the first network of the chosen organization as the scope for later operations. It is fine to re-use presuming that your organization only contains a single network. Otherwise, you would want to review the organizations object declared and printed below to review its contents.

In [ ]:
networks = dashboard.organizations.getOrganizationNetworks(organizationId=firstOrganizationId)
print('Networks:')
printj(networks)

firstNetworkId = networks[0]['id']
firstNetworkName = networks[0]['name']

# Print a blank line for legibility before showing the firstNetworkId
print('')

print(f'The firstNetworkId is {firstNetworkId}, and its name is {firstNetworkName}.')

Now that we've got the organization and network values figured out, we can get to the ask at hand:

> Show me a list of all the offline switches in a network.

The `getOrganizationDevicesStatuses` endpoint will return the devices (switches and otherwise) and their statuses, but it will not return their model numbers. To get that info, we use `getOrganizationDevices`. 

So first, we create the `devices` list. Then we create a list of those devices' statuses in `device_statuses`. Then, we use a [list comprehension](https://www.datacamp.com/community/tutorials/python-list-comprehension) to find all instances of switches in the `devices` list and put them in a new list, `devices_switches`.  

In [ ]:
devices = dashboard.organizations.getOrganizationDevices(organizationId=firstOrganizationId)
devices_statuses = dashboard.organizations.getOrganizationDevicesStatuses(organizationId=firstOrganizationId)

# Let's get the switch devices list
devices_switches = [i for i in devices if 'MS' in i['model']]
print('These are the switches:')
printj(devices_switches)

# Make a new list of all the serials from devices_switches
devices_switches_serials = [i['serial'] for i in devices_switches]

Great! Now we have a list of all the switches and their statuses `devices_switches`, as well a separate list of just their serials `devices_switches_serials`, but we only want the ones that are offline. So here, we'll use more list comprehensions to narrow down the list and create a new list with only the information we need.

In [ ]:
# Now we can list out the statuses (and whatever metadata we need) for these switches
print('We can list out the statuses (and whatever metadata we need) for these switches:')
devices_statuses_switches = [{'serial': i['serial'], 'status':i['status']} for i in devices_statuses if i['serial'] in devices_switches_serials]
print('Switch statuses are:')
printj(devices_statuses_switches)

# Print a blank line for legibility
print('')

# We can narrow it down to the ones that are offline
print('We can narrow it down to the ones that are offline:')
devices_statuses_switches_offline = [i for i in devices_statuses_switches if i['status'] != 'online']
print('Offline switches are:')
printj(devices_statuses_switches_offline)

We started from the devices list, and the devices statuses list, and created a list of offline switches. Now let's look at when they last reported to the Dashboard.

In [ ]:
# Another list comprehension!
devices_last_reported_times = [{'serial': i['serial'], 'lastReportedAt': i['lastReportedAt']} for i in devices_statuses if i['serial'] in devices_switches_serials]

printj(devices_last_reported_times)

# Final steps

Excellent, now we have a list of offline switches, and it's pretty easy to read. But what if we could use this in an Excel format? Well, there's a Python module for that, too. In this case, we're using [tablib](https://pypi.org/project/tablib/).

In [ ]:
# Let's convert that JSON-formatted data to a tabular dataset. You can copy/paste this into Excel, or write additional Python to create a new Excel file entirely!
excel_formatted = tablib.import_set(json.dumps(devices_last_reported_times), format = 'json')

# Let's see how it looks!
print(excel_formatted)

# Final thoughts

And we're done! Hopefully you found this a useful demonstration of just a few things that are possible with Meraki's Python SDK. These additional resources may prove useful along the way.

[Meraki Interactive API Docs](https://developer.cisco.com/meraki/api-v1/#!overview): The official (and interactive!) Meraki API and SDK documentation repository on DevNet.

[VS Code](https://code.visualstudio.com/): An excellent code editor with full support for Python and Python notebooks.

[Automate the Boring Stuff with Python](https://automatetheboringstuff.com/): An excellent learning resource that puts the real-world problem first, then teaches you the Pythonic solution along the way.